In [3]:
import cv2
import numpy as np
from skimage import measure, color, io
import io
import pandas as pd
from skimage import img_as_ubyte, exposure
from skimage.measure import regionprops_table
import matplotlib.pyplot as plt
from utils.nucStitch import fullNucImage16
import os
import skimage
from PIL import Image
from inducedRainbow.waterShedSegmentation import watershed
from os import listdir
from os.path import isfile, join
from utils.visualization import plotMasks

In [4]:
thresholdedInput = "/Volumes/Aorta/aorta/preprocessedImages/rainbow/inducedRainbow/thresholded/orange/0-5"
nucPath = "/Volumes/Aorta/aorta/segmentations/inducedRainbow/final/nuclei"
savingPath = '/Volumes/Aorta/aorta/segmentations/inducedRainbow/old/orange2'
stitchingPath = "/Volumes/Aorta/aorta/preprocessedImages/rainbow/inducedRainbow/stitched"
channel = "orange"
distTransformMultiplyer = 0.01

In [5]:
folders = [x[0] for x in os.walk(thresholdedInput)]
folders = sorted(folders)

In [ ]:
for folder in folders:
    os.chdir(folder)
    files = [f for f in listdir(folder) if isfile(join(folder, f)) if ".DS_Store" not in f if "8bit.tif" in f if "._" not in f]
    for file in files:
        aorta = folder.split("/")[-1]
        age = folder.split("/")[-2]
        nucleiPath = join(nucPath, age, aorta)
        savePath = join(savingPath, age, aorta)
        stitchPath = join(stitchingPath, age, aorta)
        imagePath = join(folder, file)

        if not os.path.exists(savePath):
            os.makedirs(savePath)

        npy_file = "clusterSize_" + channel + "_" + age + "_" + aorta + "_50ThresCellpose.npy"
        cellposeFileName = "cellposeSegm_" + channel + "_" + age + "_" + aorta + "_50ThresCellpose.npy"
        imageSavename = "numbersSegmentationImage_" + channel + "_" + age + "_" + aorta + "_50Thres_Cellpose.tif"
        npy_fileMarkers = "markersImage_" + channel + "_" + age + "_" + aorta + "_50Thres_Cellpose.npy"

        df = pd.DataFrame()
        nuclei = np.load(join(nucleiPath, "Cropped_IMG-1_seg.npy"), allow_pickle=True).item()['masks']
        yLength = nuclei.shape[0]
        xLength = nuclei.shape[1]
        for i in range(16):
            number = i+1
            nuclei = np.load(join(nucleiPath, "Cropped_IMG-" + str(number) + '_seg.npy'), allow_pickle=True).item()['masks']
            row = int(i/4) + 1
            column  = (i%4) + 1
            prop_dict = regionprops_table(nuclei, properties = ('label', 'centroid'))
            df_help = pd.DataFrame(prop_dict)
            df_help['picture_id'] = number
            df_help['yCoord'] = df_help['centroid-0']*row
            df_help['xCoord'] = df_help['centroid-1']*column
            df = pd.concat([df, df_help], axis=0)
        nucImage = fullNucImage16(nucleiPath)
        nucleiFiltered = np.copy(nucImage)
        markersNew = watershed(imagePath, distTransformMultiplyer)

        # Count nuclei in segmentation and check if nuclei is mostly in segmentation
        accuracy_value = 0.5
        #get rid of outlines and different backgrounds
        markersAnalyse = np.copy(markersNew)
        #markersAnalyse[markersAnalyse == -1] = 0
        nucAnalyze = np.copy(nucleiFiltered)

        clusters = list()
        dic = {}
        for i in np.unique(markersAnalyse):
            if (i != 0) & (i != -1):
                cluster = list()
                segmentationNuc = np.unique(nucAnalyze[np.where(markersAnalyse == i)])
                for j in segmentationNuc:
                    if j != 0:
                        #occurence of nucleus pixel in
                        nucOccur = np.bincount(nucAnalyze[np.where(markersAnalyse == i)])[j]
                        #How many pixels does this nucleus have:
                        coordLen = len(np.where(nucAnalyze == j)[0])
                        #check if most of nuclei values are in segmentation
                        if(nucOccur >= accuracy_value*coordLen):
                            cluster.append(j)
                if cluster:
                    dic[i] = cluster
                    clusters.append(cluster)
                else:
                    markersAnalyse[markersAnalyse == i] = 0

        prop_dict = regionprops_table(markersAnalyse, properties = ('label', 'centroid'))
        dfRainbow = pd.DataFrame(prop_dict)

        #create a new nice image
        minus1 = np.where(markersAnalyse == -1)
        for i in range(len(minus1[0])):
            minus1y = minus1[0][i]
            minus1x = minus1[1][i]
            if (minus1y != 0) & (minus1y != (markersAnalyse.shape[0] -1)) & (minus1x != 0) & (minus1x != (markersAnalyse.shape[1]-1)):
                numbers = np.unique([markersAnalyse[minus1y - 1,minus1x], markersAnalyse[minus1y + 1,minus1x], markersAnalyse[minus1y,minus1x-1], markersAnalyse[minus1y,minus1x +1]])
                uniqueMasks  = set(numbers) - set([-1, 0])
                masks = list(uniqueMasks)
                if len(masks) == 0:
                    markersAnalyse[minus1y,minus1x] = 0
        img1Filtered = cv2.imread(imagePath)
        img1Filtered[markersAnalyse == -1] = [0,255,255]

        fig, ax = plt.subplots(figsize=(36,15))
        ax.imshow(img1Filtered)
        for i in range(len(dfRainbow)):
            ax.text(dfRainbow['centroid-1'][i], dfRainbow['centroid-0'][i], dfRainbow['label'][i], fontsize=9, color = 'red')
        plt.show()

        fig, ax = plt.subplots(figsize=(72,30))
        ax.imshow(img1Filtered)
        for i in range(len(dfRainbow)):
            ax.text(dfRainbow['centroid-1'][i], dfRainbow['centroid-0'][i], dfRainbow['label'][i], fontsize=9, color = 'red')
        png1 = io.BytesIO()
        fig.savefig(png1, format="png")
        png2 = Image.open(png1)
        png2.save(join(savePath, imageSavename))
        png1.close()

        png1.close()
        os.chdir(savePath)
        np.save(npy_file, dic)

        markersdic = {}
        markersdic['image'] = markersAnalyse
        np.save(npy_fileMarkers,  markersdic)

        cellposeFile = {}
        #read in image
        fileRed = [f for f in listdir(stitchPath) if isfile(join(stitchPath, f)) if ".DS_Store" not in f if "orange.tif" in f if "._" not in f]
        fileNuc = [f for f in listdir(stitchPath) if isfile(join(stitchPath, f)) if ".DS_Store" not in f if "nuclei.tif" in f if "._" not in f]
        originalImg = skimage.io.imread(join(stitchPath, fileRed[0]))
        originalImg = np.where(originalImg < np.quantile(originalImg, 0.01), np.quantile(originalImg, 0.01),
                                  (np.where(originalImg > np.quantile(originalImg, 0.99), np.quantile(originalImg, 0.99), originalImg)))
        nucleiImg = skimage.io.imread(join(stitchPath, fileNuc[0]))
        zeros = np.zeros(originalImg.shape, dtype="uint8")
        #8bit normalize & merge
        dapi8bitNormalized= img_as_ubyte(exposure.rescale_intensity(nucleiImg))
        rainbow8bitNormalized = img_as_ubyte(exposure.rescale_intensity(originalImg))
        dapi8bitNormalized = np.asarray(dapi8bitNormalized)
        dapi8bitNormalized = dapi8bitNormalized*0.1
        dapi8bitNormalized = dapi8bitNormalized.astype(rainbow8bitNormalized.dtype)
        merged8bitNormalized = cv2.merge([rainbow8bitNormalized, dapi8bitNormalized, zeros])
        #define masks and outlines
        masks = np.where(markersAnalyse==-1, 0, markersAnalyse)
        outlines = np.where(markersAnalyse==-1,1,0)
        #save in dic
        cellposeFile['masks'] = masks
        cellposeFile['img'] = merged8bitNormalized
        cellposeFile['outlines'] = outlines
        np.save(join(savePath, cellposeFileName),cellposeFile)